In [ ]:
from bs4 import BeautifulSoup
import urllib
from selenium import webdriver
from time import sleep
import pandas as pd
import numpy as np

In [ ]:
job_urls = []
vPath = './chromedriver/chromedriver'

In [ ]:
driver = webdriver.Chrome(executable_path=vPath)

# just scrape everything that is related first
url_lst = ["https://www.mycareersfuture.sg/search?search=data%20analyst&page={}", 
           "https://www.mycareersfuture.sg/search?search=data%20scientist&page={}", 
           "https://www.mycareersfuture.sg/search?search=analytics&page={}", 
           "https://www.mycareersfuture.sg/search?search=data%20engineer&page={}", 
           "https://www.mycareersfuture.sg/search?search=machine%20learning&page={}", 
           "https://www.mycareersfuture.sg/search?search=business%20intelligence&page={}",
           "https://www.mycareersfuture.sg/search?search=business%20analyst&page={}", 
           "https://www.mycareersfuture.sg/search?search=artificial%20intelligence&page={}",
           "https://www.mycareersfuture.sg/search?search=ai&page={}",
           "https://www.mycareersfuture.sg/search?search=data&page={}" 
          ]
           
for u in url_lst:
    i = 0
    for i in range(0,30):
        driver.get(u.format(i))
        sleep(5) 
        html = BeautifulSoup(driver.page_source, 'lxml')
        for job_card in html.find_all('div', id=lambda x: x and x.startswith('job-card-')):
            job_urls.append('https://www.mycareersfuture.sg'+job_card.find('a')['href'])
      
    
driver.close()

In [ ]:
len(job_urls)


In [ ]:
job_urls2 = set(job_urls)

In [ ]:
#save urls as csv
pd.DataFrame(list(job_urls2)).to_csv('jobs_url.csv', encoding='utf-8')

In [ ]:
job_df = pd.read_csv('jobs_url.csv')
job_ls = job_df.iloc[:,1].values

In [ ]:
# loop to get details
companies = []
titles = []
emp_type = []
seniority = []
category = []
salaries = []
responsibilities = []
requirements = []
failed_url = []

# loop through to get all the needed info, takes a while...
driver = webdriver.Chrome(executable_path=vPath)
for ind, url_job in enumerate(job_ls): 
    try:
        driver.get(url_job)
        sleep(5)
        html_job = driver.page_source
        soup_job = BeautifulSoup(html_job, 'lxml')
        companies.append(soup_job.find('p', {'name': 'company'}))
        titles.append(soup_job.find('h1', {'id': 'job_title'}))
        emp_type.append(soup_job.find('p', {'id': 'employment_type'}))
        seniority.append(soup_job.find('p', {'id': 'seniority'}))
        category.append(soup_job.find('p', {'id': 'job-categories'}))
        salaries.append(soup_job.find('div', {'class': 'lh-solid'}))
        responsibilities.append(soup_job.find('div',{'id':'job_description'}))
        requirements.append(soup_job.find('div',{'id':'requirements'}))
    except:
        failed_url.append(url_job)

driver.close()

In [ ]:
info = [companies, titles, emp_type, seniority, category, salaries, responsibilities, requirements]


In [ ]:
clean = {}
for ind, ls in enumerate(info):
    clean[ind] = []
    for x in ls:
        try:
            clean[ind].append(x.text)
        except:
            clean[ind].append(np.nan)


In [ ]:
#convert dictionary into dataframe and rename columns
df = pd.DataFrame(clean)
df.columns = ['Company', 'Title', 'Emp_type', 'Seniority', 'Industry', 'Salary', 'Responsibility', 'Requirements']
df

In [ ]:
#export
df.to_csv('cf_jobsdf.csv', encoding='utf-8')